Maya Asher, 4/8/24
# Analyzing the SBCSAE
**NEW CONTINUING**

Using the processed data, which can be found [here](https://github.com/Data-Science-for-Linguists-2024/Stance-Taking-in-Spontaneous-Speech/blob/main/reprocessing_sbcsae.ipynb), we can begin our stance taking anaylsis.

## Import

In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%pprint

Pretty printing has been turned OFF


## Unpickle

In [2]:
f = open('df2col.pkl', 'rb')
df2c = pickle.load(f)
f.close()

In [3]:
f = open('df3col.pkl', 'rb')
df3c = pickle.load(f)
f.close()

In [4]:
f = open('df4col.pkl', 'rb')
df4c = pickle.load(f)
f.close()

In [7]:
# got them all!
len(df4c) + len(df3c) + len(df2c)

43

## Object of analysis